# Find Pyramids
There are some example images in the `./images` folder, these can be selected using the following string.  

In [1]:
image_name = '39-31 top 5kx.tif' # done
#image_name = '39-58 top 5kx.tif' # done
#image_name = '39-65 top 5kx.tif' # done
#image_name = '39-70 top 5kx.tif'

## Measure pixel size
Here we load the image data we want to analyze and do our first measurements. I have measured the size of the scalebar in pixels to calibrate the image axes. 

In [2]:
%matplotlib qt 
import matplotlib.pyplot as plt
import numpy as np

from skimage.io import imread

imdata = imread('./images/'+image_name) / 2**8 # uint8 to float in range 0-1
f, ax = plt.subplots()
im = ax.imshow(imdata, cmap=plt.cm.gray)
plt.colorbar(im, None, ax)

In [3]:
S  = 5.     # scalebar size in µm
x1 = 989    # right pixel of the scale bar
x0 = 780    # left ...
ps = S / (x1 - x0)

print('Pixel Size = {0:.5f} µm / pixel'.format(ps))

Pixel Size = 0.02392 µm / pixel


## Automatic foregound peak finding
This scripts aims to find the local maxima in the image we just loaded. It uses some filtering steps to achieve this. It only achieves partial success in finding the maxima. In the next step we will correct the result by hand. 

This script:
- Finds the foreground automatically.
- Uses brute force to find ALL local maxima in the foreground.
- Reduces this result to single isolated indices.
- Does the initial watershed segmentation to obtain the color pyramids. 

In [4]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage as ndi
from skimage.io import imread
from skimage.morphology import disk
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.filters import threshold_otsu

def foreground_detection(imdata, threshold_factor = 1., *args, **kwargs):
    
    # calculate threshold
    threshold = threshold_otsu(imdata) * threshold_factor
    
    # Remove small objects
    mask = imdata > threshold
    mask = ndi.binary_erosion(mask, *args, **kwargs)
    mask[0,:] = 0
    mask[:,1] = 0
    mask = ndi.binary_dilation(mask, *args, **kwargs)
    
    return mask, threshold

# Load image: use scikit-image for IO and remove speckle noise
imdata = imread('./images/'+image_name)[:750, :] / 2**8 
imdata = ndi.median_filter(imdata, size=2)

# Foreground detection: apply threshold, remove small objects and label
structure  = disk(2)
iterations = 5
foreground_mask, threshold = foreground_detection(
    imdata = imdata, 
    threshold_factor = 1., 
    structure=structure, 
    iterations=iterations, )
foreground_labels, foreground_Nlabels = ndi.label(foreground_mask)

# find local max: build mask from local maxima in the foreground
iterations = 4
maxima_mask = peak_local_max(
    image = imdata,
    labels = foreground_labels,
    num_peaks_per_label=25,
    min_distance=1, 
    indices=False, )

# concentrate and characterize the local max
maxima_mask = ndi.binary_dilation(maxima_mask, iterations=iterations)
maxima_markers = np.argwhere(maxima_mask)
maxima_labels, maxima_Nlabels = ndi.label(maxima_mask)

# Run again find local max for cleanup
maxima_mask = peak_local_max(
    image = imdata,
    labels = maxima_labels,
    num_peaks_per_label=1,
    min_distance=1, 
    indices=False, )

# concentrate and characterize the local max
maxima_markers = np.argwhere(maxima_mask)
maxima_labels, maxima_Nlabels = ndi.label(maxima_mask)

# watershed to fill the basins 
watershed_labels = watershed(-imdata, maxima_labels, watershed_line=False)
# if watershed_line=True: watershed_line = np.nonzero(watershed_labels==0)

## Interactive plot
In this step and interactive plot allows to quickly correct the result. If the previous step was done correctly, most of the larger pyramid apices have been automatically found. Some of the more intense could show 2 or more maxima which may be wrong. Also small pyramids are very hard to automatically find. The interactive plot has some built in functionality to manually edit the result:

- Using the Left / Right click buttons we can Add / Remove points
- The watershed segmentation is repeated every time a point is Added / Removed (colors are updated accordingly).

When we are done the result can be saved. A load command is also provided to retrieve saved results.

Note: If the plot is closed, start again from this point.

In [9]:
figsize = plt.figaspect(1/2.)
f, axs = plt.subplots(1, 2, dpi=150, figsize=figsize, sharex=True, sharey=True)

Ny, Nx = imdata.shape
x, y = np.arange(Nx), np.arange(Ny)

ax = axs[0]
ax.imshow(imdata, cmap='afmhot')
contours = ax.contour(x, y, foreground_mask, [0.5], linewidths=0.5, colors='g')
#ax.contour(x, y, maxima_mask, [0.5], linewidths=0.5, colors='r')
pc_markers = ax.scatter(maxima_markers[:, 1], maxima_markers[:, 0], s=1, c='r', marker='o')
ax.axis('off')

ax = axs[1]
ax.imshow(imdata, cmap=plt.cm.gray)
img_watershed = ax.imshow(watershed_labels, cmap=plt.cm.flag, alpha=.2)
ax.axis('off')

# crossairs
axvlines = []
axhlines = []
for ax in axs.flat:
    #plt.colorbar(ax.images[0], None, ax)
    axvlines += [ax.axvline(x=0., c='g', ls=':', lw=0.5),]
    axhlines += [ax.axhline(y=0., c='g', ls=':', lw=0.5),]

def onMouseMove(event):
    if not event.inaxes:
        return
    x = event.xdata if event.xdata is not None else 0.
    y = event.ydata if event.ydata is not None else 0.
    for axvl, axhl in zip(axvlines, axhlines):
        axvl.set_xdata(x)
        axhl.set_ydata(y)
    f.canvas.draw()

f.canvas.mpl_connect('motion_notify_event', onMouseMove)

def addPoint(path_collection, new_point, c='b'):
    offsets = path_collection.get_offsets()
    facecls = path_collection.get_facecolors()
    if facecls.shape[0] == 1:
        facecls = np.repeat(facecls, offsets.shape[0], axis=0)
    offsets = np.concatenate([offsets, np.array(new_point, ndmin=2)])
    facecls = np.concatenate([facecls, np.array(plt.matplotlib.colors.to_rgba(c), ndmin=2)])
    path_collection.set_offsets(offsets)
    path_collection.set_facecolors(facecls)
    path_collection.axes.figure.canvas.draw_idle()
    
def rmPoint(path_collection, coordinates):
    offsets = path_collection.get_offsets()
    facecls = path_collection.get_facecolors()
    if facecls.shape[0] == 1:
        facecls = np.repeat(facecls, offsets.shape[0], axis=0)
    idx_closest=np.argmin(np.sum((offsets-np.array(coordinates, ndmin=2))**2., axis=1))
    offsets = np.delete(offsets, idx_closest, axis=0)
    facecls = np.delete(facecls, idx_closest, axis=0)
    path_collection.set_offsets(offsets)
    path_collection.set_facecolors(facecls)
    path_collection.axes.figure.canvas.draw_idle()
    
def onClick(event):
    if not event.inaxes:
        return
    if event.button == 1:
        addPoint(pc_markers, (event.xdata, event.ydata))
    elif event.button ==3:
        rmPoint(pc_markers, (event.xdata, event.ydata))
    repeat_watershed()

def repeat_watershed():
    # repeat watershed
    offsets = pc_markers.get_offsets()
    new_markers = np.round(offsets[:, ::-1], decimals=0)
    new_markers = new_markers.astype('int')
    new_mask = np.zeros_like(maxima_mask, dtype=bool)
    new_markers = np.ravel_multi_index(new_markers.T, new_mask.shape)
    np.ravel(new_mask)[new_markers] = True
    new_labels, new_Nlabels = ndi.label(new_mask)
    watershed_labels = watershed(-imdata, new_labels)
    img_watershed.set_data(watershed_labels)
    img_watershed.set_clim(0, new_Nlabels)

def load_path_collection(file_name, path_collection, c='y'):
    offsets = np.load(file_name)
    facecls = np.repeat(
        np.array(plt.matplotlib.colors.to_rgba(c))[None, :], 
        offsets.shape[0], 
        axis=0, )
    path_collection.set_offsets(offsets)
    path_collection.set_facecolors(facecls)
    path_collection.axes.figure.canvas.draw_idle()  
    repeat_watershed()

def save_path_coordinates(file_name, path_collection):
    coordinates = path_collection.get_offsets()
    coordinates = np.asarray(coordinates, np.float32)
    np.save(file_name,coordinates) 

cid = f.canvas.mpl_connect('button_press_event', onClick)

# some cheap scalebar
scalebar_size = 5 # µm
scalebar_size_pix = scalebar_size / ps
ax = axs[0]
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
scalebar = AnchoredSizeBar(
    transform = ax.transData,
    size = scalebar_size_pix, 
    label = '{0:d} µm'.format(int(round(scalebar_size))),
    loc = 'lower left', 
    pad = 1,
    color = 'white',
    frameon = False,
    label_top = True,
    size_vertical = 8, )

ax.add_artist(scalebar)

f.tight_layout()

### Save the result
**Use with caution, can overwrite important data**

In [14]:
save_name = image_name.replace('.tif',' - pyramid coords.npy')
#save_path_coordinates(sname, pc_markers)

### Load result

In [10]:
save_name = image_name.replace('.tif',' - pyramid coords.npy')
load_path_collection(save_name, pc_markers)

## Measure pyramid vertices

In [26]:
watershed_labels = img_watershed.get_array()

# Run again find local max for cleanup
watershed_center_indices = peak_local_max(
    image = imdata,
    labels = watershed_labels,
    num_peaks_per_label=1,
    min_distance=1, 
    indices=True, )

ax = axs[1]
wm_markers = ax.scatter(watershed_center_indices[:, 1], watershed_center_indices[:, 0], s=1, c='w', marker='x')
ax.axis('off')

(-0.5, 999.5, 749.5, -0.5)

In [27]:
from skimage.measure import regionprops
from skimage.segmentation import find_boundaries
#from skimage.segmentation import clear_border, mark_boundaries

# pixels at the border
boundaries = find_boundaries(watershed_labels)

# maxima in the watershed as array
watershed_max = peak_local_max(
    image = imdata,
    labels = watershed_labels,
    num_peaks_per_label=1,
    min_distance=1, 
    indices=False, )

# pyramid angles
#pyangle = np.pi / 4 
pyangle = np.deg2rad(45.)
pyangle_n = np.arange(4) * np.pi / 2 + pyangle
pyangle_n = pyangle_n - np.pi 

labels = np.unique(watershed_labels)
Nlabels = len(labels)
xpos = np.zeros([Nlabels, 4, 2])
xdis = np.zeros([Nlabels, 4, 2])
lines  = []
for io, label in enumerate(labels):
    # find the border pixels
    label_mask = watershed_labels == label
    label_bounds = boundaries.copy()
    label_bounds[~label_mask] = 0
    bounds = np.argwhere(label_bounds)

    # find the center pixel
    label_max = watershed_max.copy()
    label_max[~label_mask] = 0
    center = np.argwhere(label_max)

    # get the angles
    distances = bounds - center
    angles = np.arctan2(distances[:, 1], distances[:, 0])
    coords = []
    dists = []
    for ai in pyangle_n:
        idx = np.argmin(np.abs(angles - ai))
        coord = bounds[idx]
        coords.append(coord)
        dist = distances[idx]
        dists.append(dist)
        
        line = plt.Line2D(
            xdata = [coord[1], center[0, 1]], 
            ydata = [coord[0], center[0, 0]],
            linestyle= '--',
            linewidth= 0.5, )
        lines.append(line)
        
    coords = np.array(coords)
    xpos[io, ...] = coords
    
    dists = np.array(dists)
    xdis[io, ...] = dists

ax = axs[1]
for line in lines:
    ax.add_artist(line)
    
#f, ax = plt.subplots(dpi=150.)
#xray_plt = xpos.reshape(Nlabels*4, 2)
#ax.imshow(imdata)
#ax.imshow(boundaries, alpha=0.5)
#ax.scatter(watershed_center_indices[:, 1], watershed_center_indices[:, 0], s=5, c='k', marker='x')
#ax.scatter(xray_plt[:, 1], xray_plt[:, 0], s=5, c='r', marker='+')
#for line in lines:
#    ax.add_line(line)

In [28]:
sname = image_name.replace('.tif',' - vertex coords.npy')
np.save(sname, xdis)

In [29]:
sname = 'imagenes/'+image_name.replace('.tif', ' - find pyramids.png')
f.savefig(sname, facecolor='none', dpi=150.)

## Histogram vertex properties

In [30]:
#image_name = '39-31 top 5kx.tif' # done
#image_name = '39-58 top 5kx.tif' # done
#image_name = '39-65 top 5kx.tif' # done
image_name = '39-70 top 5kx.tif'

In [31]:
ps = 0.02392 # pixel-size µm / pixel

In [37]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

sname = image_name.replace('.tif',' - vertex coords.npy')
vertices = np.load(sname)
moduli = np.sqrt(np.sum(vertices**2, -1)) * ps
angles = np.arctan2(vertices[..., 1], vertices[..., 0])
max_indices = np.argmax(moduli, -1)
moduli_max = np.array([moduli[io, idx] for io, idx in enumerate(max_indices)])
angles_max = np.array([angles[io, idx] for io, idx in enumerate(max_indices)])
angles_max_deg = np.rad2deg(angles_max+np.pi)
angles_deg = np.rad2deg(angles+np.pi)

v_plt = vertices.reshape(-1, 2)

figsize = plt.figaspect(1)
f, ax = plt.subplots(dpi=150., figsize=figsize)
ax.scatter(v_plt[:, 1]*ps, v_plt[:, 0]*ps, s=5, c='b', marker='+')
ax.set_ylim(-3., +3.)
ax.set_xlim(-3., +3.)
ax.grid('on')
ax.set_aspect('equal')
ax.set_title('Vertex coordinates [µm]')
sname = 'imagenes/'+image_name.replace('.tif',' - vertex coords.png')
f.savefig(sname)

figsize = plt.figaspect(1/2.5)
f, axs = plt.subplots(1, 2, dpi=150., figsize=figsize)

ax = axs[0]
ax.hist(moduli.reshape(-1), bins=50, range=(0., 4.5), label='All vertices')
ax.hist(moduli_max, bins=25, range=(0., 4.5), color='r', label='Max vertices')
ax.set_xlabel('Vertex size [µm]')
ax.set_ylabel('Counts [pyramid]')
ax.legend()

ax = axs[1]
ax.hist(angles_deg.reshape(-1), bins=50, range=(0., 360.))
ax.hist(angles_max_deg, bins=50, range=(0., 360.), color='r')
ax.set_xlabel('Vertex orientation [deg]')

sname = 'imagenes/'+image_name.replace('.tif',' - vertex stats.png')
f.savefig(sname)

## Histogram vertex properties - height

In [34]:
#image_name = '39-31 top 5kx.tif' # done
#image_name = '39-58 top 5kx.tif' # done
#image_name = '39-65 top 5kx.tif' # done
image_name = '39-70 top 5kx.tif'

In [35]:
ps = 0.02392 # pixel-size µm / pixel
angle = 54.7

In [36]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np

sname = image_name.replace('.tif',' - vertex coords.npy')
vertices = np.load(sname)
moduli = np.sqrt(np.sum(vertices**2, -1)) * ps
height = np.tan(np.deg2rad(angle)) * moduli
max_indices = np.argmax(moduli, -1)
height_max = np.array([height[io, idx] for io, idx in enumerate(max_indices)])


figsize = plt.figaspect(0.8)
f, ax = plt.subplots(dpi=150., figsize=figsize)

Rbins = (0., 6.)
ax.hist(height.reshape(-1), bins=50, range=Rbins, label='All vertices')
ax.hist(height_max, bins=25, range=Rbins, color='r', label='Max vertices')
ax.set_xlabel('Pyramid height [µm]')
ax.set_ylabel('Counts [pyramid]')
ax.legend()

sname = 'imagenes/'+image_name.replace('.tif',' - Pyramid height.png')
f.savefig(sname)

In [28]:
plt.close('all')

## Export vertex sizes and orientations to excel tables

In [38]:
import numpy as np

ps = 0.02392 # pixel-size µm / pixel

image_names = [
    '39-31 top 5kx.tif', 
    '39-58 top 5kx.tif', 
    '39-65 top 5kx.tif', 
    '39-70 top 5kx.tif', ]

angle = 54.7

for name in image_names:
    lname = name.replace('.tif',' - vertex coords.npy')
    vertices = np.load(lname)
    moduli = np.sqrt(np.sum(vertices**2, -1)) * ps
    angles = np.arctan2(vertices[..., 1], vertices[..., 0])
    max_indices = np.argmax(moduli, -1)
    moduli_max = np.array([moduli[io, idx] for io, idx in enumerate(max_indices)])
    angles_max = np.array([angles[io, idx] for io, idx in enumerate(max_indices)])
    angles_max_deg = np.rad2deg(angles_max+np.pi)
    height_max = np.tan(np.deg2rad(angle))*moduli_max
    sname = name.replace('.tif', '- vertex module.csv')
    np.savetxt(
        fname = sname, 
        X = np.stack([moduli_max, angles_max_deg, height_max], axis=1),
        fmt = '%.5f',
        delimiter = ',',
        header='# Modulus [um], Orientation [deg], Height [um]', 
        comments='# Original image name: {0:s} \n# Pixel Size = {1:.5f} um / pixel \n'.format(name, ps), )